In [1]:
import pandas as pd
import numpy as np

### Metaphysical Description

In [2]:
# Get metaphysical description
fname = "data/input/meanings.csv"
meanings = pd.read_csv(fname)[['stone', 'description']]

In [3]:
print(meanings.shape)
meanings.head()

(100, 2)


,stone,description
0,agate,"Agates are earth stones. Overall, agates are g..."
1,alexandrite,"Rather than just bring you joy, Alexandrite he..."
2,amazonite,Amazonite helps to harmonize apparently differ...
3,amber,"Amber carries the energy of the sun, warmth, a..."
4,amethyst,Amethyst is a powerhouse stone. It vibrates at...


### Hardness

In [4]:
# Get hardness
fname = "data/input/hardness.csv"
hardness = pd.read_csv(fname)

In [5]:
print(hardness.shape)
hardness.head()

(346, 2)


,stone,hardness
0,diamond,10.0
1,synth. moissanit,9.5
2,sapphire,9.0
3,ruby,9.0
4,yag,8.5


In [6]:
# merge data
df = meanings.merge(hardness, on='stone', how='outer')

#df = df.merge(hardness2, on='stone', how='left') # keeping just 100 stones w meanings
#df['hardness'] = [i.hardness_y if i.hardness_y > 0.0 else i.hardness_x for _, i in df.iterrows()]
#df = df[["stone", "description", "hardness", "diaphaneity", "refractive_idx", "density"]]
#df = df[df.hardness > 0]

In [7]:
print(df.shape)
df.head()

(394, 3)


,stone,description,hardness
0,agate,"Agates are earth stones. Overall, agates are g...",NaN
1,alexandrite,"Rather than just bring you joy, Alexandrite he...",8.50
2,amazonite,Amazonite helps to harmonize apparently differ...,6.25
3,amber,"Amber carries the energy of the sun, warmth, a...",2.25
4,amethyst,Amethyst is a powerhouse stone. It vibrates at...,7.00


### Chakras!

In [8]:
fname_chakra = "data/input/chakra_data.csv"
chakras_ref = pd.read_csv(fname_chakra)
chakras_ref = chakras_ref.sort_values('Position').reset_index(drop=True)
# make crystals a list
chakras_ref.Crystals = [i.split('; ') for i in chakras_ref.Crystals]

In [9]:
# Add stones from chakras to chakra_ref
fname_chakra_stones = "data/input/chakra_stones.csv"
chakras = pd.read_csv(fname_chakra_stones)
chakras.chakra = [[int(j) for j in i.split(" & ")] for i in chakras.chakra]
for idx, (stone, chakra) in chakras.iterrows():
    for k in chakra:
        chakras_ref.iloc[k-1,7].append(stone)
# remove duplicates
chakras_ref.Crystals = [list(set(i)) for i in chakras_ref.Crystals]
chakras_ref.Properties = [list(set(i.split("; "))) for i in chakras_ref.Properties]

In [10]:
chakras_ref[:1]

,Name_Eng,Position,Body_Location,Name_San,Color,Mantra,Properties,Crystals,Resonant_Frequency_Hz,Sound,Note,Translation_San
0,Root,1,spine; base of tailbone,Muladhara,Red,I am,"[comfort, security, protection, safety, ground...","[black tourmaline, garnet, crystal quartz, obs...",432,Ay-Way,A,base or support of root


In [11]:
# Add stones from chakras_ref to chakras
for idx, i in chakras_ref.iterrows():
    for j in i[7]:
        p = chakras.chakra[chakras.stone == j]
        if len(p):
            chakras.iloc[p.index[0], 1].append(i[1])
        else:
            chakras = pd.concat([chakras, pd.DataFrame([{'stone':j, 'chakra':[i[1]]}])], ignore_index=True)
chakras.chakra = [i for i in chakras.chakra]
# remove duplicates
chakras.chakra = [list(set(i)) for i in chakras.chakra]
chakras = chakras.sort_values('stone').reset_index(drop=True)

In [12]:
# This is how many stones I have chakra data for from "chakra_data.csv" and "chakra_stones.csv"
print(chakras.shape)
chakras.head()

(53, 2)


,stone,chakra
0,amazonite,"[4, 5]"
1,amethyst,"[6, 7]"
2,ametrine,[6]
3,angelite,[5]
4,apatite,[5]


In [32]:
# add chakras to our original df
df_full = chakras.merge(df, on='stone', how='outer')
df_full = df_full.sort_values("stone", ascending=True).reset_index(drop=True)
#chakras_full = df_full[~df_full.chakra.isna()]

In [33]:
print(df_full.shape)
df_full.head()

(408, 4)


,stone,chakra,description,hardness
0,actinolite,NaN,NaN,5.75
1,adamite,NaN,NaN,3.50
2,aegirine,NaN,NaN,6.00
3,aeschynite,NaN,NaN,5.50
4,agate,NaN,"Agates are earth stones. Overall, agates are g...",NaN


### Physical Descriptions

In [56]:
# Data from: https://beadage.net/gemstones
fname = "data/output/phys_desc.csv"
phys_desc = pd.read_csv(fname, sep=",")
phys_desc.shape

(96, 2)

In [53]:
df_final = df_full.merge(phys_desc, on="stone", how="outer").sort_values('stone').reset_index(drop=True)

In [55]:
print(df_final.shape)
df_final.head()

(414, 5)


,stone,chakra,description,hardness,physical_description
0,actinolite,NaN,NaN,5.75,NaN
1,adamite,NaN,NaN,3.50,NaN
2,aegirine,NaN,NaN,6.00,NaN
3,aeschynite,NaN,NaN,5.50,NaN
4,agate,NaN,"Agates are earth stones. Overall, agates are g...",NaN,NaN


### Export Data for App
Columns: Name	Chakra	Hardness	Physical_Description	Metaphysical_Properties

Total records: 414
    
    w/ chakra: 53
    w/ metaphys desc: 100
    w/ hardness: 346
    w/ physical desc: 96

In [58]:
fname = "data/output/crystals.csv"
df_final.to_csv(fname, index=False)

In [60]:
df_final.columns

Index(['stone', 'chakra', 'description', 'hardness', 'physical_description'], dtype='object')

In [83]:
# Get only crystals with at least two data points
counts = df_final[['stone', 'chakra', 'description', 'hardness', 'physical_description']].count(axis=1)
df_w_data = df_final.loc[counts.index[counts >= 3]].reset_index(drop=True)

In [85]:
print(df_w_data.shape)
df_w_data.head()

(97, 5)


,stone,chakra,description,hardness,physical_description
0,alexandrite,NaN,"Rather than just bring you joy, Alexandrite he...",8.50,Alexandrite is a member of the chrysoberyl fam...
1,amazonite,"[4, 5]",Amazonite helps to harmonize apparently differ...,6.25,Amazonite is a brilliant green to bluish-green...
2,amber,NaN,"Amber carries the energy of the sun, warmth, a...",2.25,"Amber is fossilized sap or pitch from trees, a..."
3,amethyst,"[6, 7]",Amethyst is a powerhouse stone. It vibrates at...,7.00,Amethyst is a purple variety of quartz crystal...
4,ametrine,[6],Ametrine combines the healing properties of Am...,NaN,Ametrine is a combination of amethyst and citr...


In [86]:
fname = "data/output/crystals97.csv"
df_w_data.to_csv(fname, index=False)

### Information Gathering

In [51]:
def get_chakra_deetz(i):
    s = "Chakra #" + str(i)
    s += ", also known as the "
    s += chakras_ref.iloc[i-1, 0] # name
    s += " chakra. It is located at the "
    s += chakras_ref.iloc[i-1, 2] # location
    s += '. In Sanskrit it is refered to as "' + chakras_ref.iloc[i-1, 3]
    s += '" and its mantra is: ' + chakras_ref.iloc[i-1,5]
    s += ". Its color is " + chakras_ref.iloc[i-1,4]
    s += " and it's related to "
    for x in chakras_ref.Properties[i-1]:
        s += x + ", "
    s += "etc... Some crystals associated with this chakra include: "
    crystals = chakras_ref.iloc[i-1, 7]
    s += crystals[0] + ", "
    for c in crystals[1:-1]:
        s += c + ", "
    s += "and " + crystals[-1] + "."
    return s
#print(get_chakra_deetz(7))

In [54]:
def get_description(stone):
    return df_full[df_full.stone == stone].description.item()
#print(get_description("emerald"))

In [93]:
def get_stones(find_chak):
    d = chakras_full[[find_chak in x for x in chakras_full.chakra]]
    d = d[['stone', 'chakra', 'description', 'hardness']]
    d.reset_index(drop=True, inplace=True)
    for _, x in d.iterrows():
        print(x[0])
        if pd.notna(x[3]):
            print("hardness: " + str(x[3]))
        print(x[2])
        print("\n")
    return d
#f = get_stones(4)

In [42]:
def get_facts(stone):
    s = stone + "\nDescription: "
    i = chakras_full[chakras_full.stone == stone]
    s += i.description.item() + "\nChakras: "
    s += str(i.chakra.item()) + "\n"
    return(s)
print(get_facts("peridot"))

NameError: name 'chakras_full' is not defined

### Future Considerations

In [ ]:
# TODO: Get pricing at least 2 labels of expensive or inexpensive
# TODO: Get Zodiac -- scrape: https://www.allcrystal.com/category/chakra/
# Scrape emotions data from: https://thecrystalcouncil.com/crystals
# Planet, Element